In [1]:
import base64
import requests
import numpy as np
import pandas as pd
import datetime
from urllib.parse import urlencode
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
client_id="60a3d3bac4a34f62...."
client_secret="2777a56841844f91..."

In [3]:
#store album id for each decade
playlist_ids=    {"album_id_10s":"37i9dQZF1DX5Ejj0EkURtP",
               "album_id_00s":"37i9dQZF1DX4o1oenSJRJd",
               "album_id_90s":"37i9dQZF1DXbTxeAdrVG2l",
               "album_id_80s":"37i9dQZF1DX4UtSsGT1Sbe",
               "album_id_70s":"37i9dQZF1DWTJ7xPn4vNaz",
               "album_id_60s":"37i9dQZF1DXaKIA8E7WcJj",
               "album_id_50s":"37i9dQZF1DWSV3Tk4GO2fq"}

In [4]:
class SpotifyAPI(object):
    access_token=None
    access_token_expires=datetime.datetime.now()
    access_token_did_expire=True
    client_id=None
    client_secret=None
    token_url ="https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        self.client_id=client_id
        self.client_secret=client_secret
        
    def get_client_credentials(self):
        client_id=self.client_id
        client_secret=self.client_secret
        if client_id==None or client_secret==None:
            raise Exception("You must set client_id and client_secret")
                
        client_creds_bytes=f"{client_id}:{client_secret}".encode() #type = bytes
        client_creds_b64=base64.b64encode(client_creds_bytes)
        return client_creds_b64.decode()
         
        
    def perform_auth(self):
        token_url=self.token_url
        token_data=self.get_token_data()
        token_headers=self.get_token_headers()
        client_creds_b64=self.get_client_credentials()
        token_headers=self.get_token_headers()
        req=requests.post(token_url, data=token_data, headers=token_headers )
        if req.status_code not in range (200,299):
            raise Exception("Could not authenticate client.")
        
        data=req.json()
        now=datetime.datetime.now()
        self.access_token=data['access_token']
        expires_in=data['expires_in']#seconds
        expires=now+datetime.timedelta(seconds=expires_in)
        self.access_token_expires=expires
        self.access_token_did_expire=now > expires
        return True
        
        
    def get_token_data(self):
        return {
            'grant_type':'client_credentials'
        }

    def get_token_headers(self):
        client_creds_b64=self.get_client_credentials()
        return {
            'Authorization':f"Basic {client_creds_b64}" 
        }
    
    def get_resource_header(self):
        access_token=self.access_token
        headers = {
            'Authorization': f"Bearer {access_token}"
        }
        return headers

    def get_access_token(self):
        token=self.access_token
        expires=self.access_token_expires
        now=datetime.datetime.now()
        if now > expires:
            self.perform_auth()
            return self.get_access_token()
        elif token==None:
            self.perform_auth()
            return self.get_access_token()
        return token
        

    def get_resource(self, lookup_id, version="v1", resource_type="album"):
        lookup_url=f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        access_token=self.access_token
        headers = {
            'Authorization': f"Bearer {access_token}"
        }
        req=requests.get(lookup_url,headers=headers)
        if req.status_code not in range (200,299):
            return {}
        return req.json()
    
    def get_album(self, lookup_id):
        return self.get_resource(lookup_id, version="v1", resource_type="album")
    
    def get_playlist_tracks(self, playlist_id, version="v1"):
        lookup_url=f"https://api.spotify.com/{version}/playlists/{playlist_id}/tracks"
        req=requests.get(lookup_url,headers=self.get_resource_header())
        if req.status_code not in range (200,299):
            return req.status_code
        return req.json() 
    

    def get_artist(self, lookup_id):
        return self.get_resource(lookup_id, version="v1", resource_type="artist")

    def get_audio_analysis(self,track_id, version="v1"):
        lookup_url="https://api.spotify.com/{version}/audio-analysis/{track_id}"
        req=requests.get(lookup_url,headers=self.get_resource_header())
        if req.status_code not in range (200,299):
            print(req.status_code)
        return req.json()
        
    def get_audio_features(self,track_id, version="v1"):
        lookup_url=f"https://api.spotify.com/{version}/audio-features/{track_id}"
        req=requests.get(lookup_url,headers=self.get_resource_header())
        if req.status_code not in range (200,299):
            print(req.status_code)
        return req.json() 
        
    def search(self, query, search_type="track"):
        access_token=self.get_access_token()
        headers = {
            'Authorization': f"Bearer {access_token}"
        }

        data=urlencode({
            'q': query,
            'type': search_type.lower()
        })
        endpoint="https://api.spotify.com/v1/search"
        lookup_url=f"{endpoint}?{data}"
        req=requests.get(lookup_url,headers=headers)
        if req.status_code not in range (200,299):
            return {}
        return req.json()

In [5]:
spotify=SpotifyAPI(client_id,client_secret)
spotify.perform_auth()

True

In [6]:
def get_playlist_track_ids(playlist_dictionary):
    playlist_track_ids = []
    # get info about tracks in playlist 
    for playlist_name, playlist_id in playlist_dictionary.items():
        playlist_track_info=spotify.get_playlist_tracks(f"{playlist_id}")
        #get ids for every track in playlist 
        for track in playlist_track_info['items']:
            playlist_track_ids.append([track['track']['uri'].split(":")[-1], #track id
                                       track['track']['artists'][0]['name'], # artist name
                                       track['track']['name'], #track name
                                       playlist_name]) #playlist name
    df_tracks =pd.DataFrame(playlist_track_ids, columns=['track_id', 'artist', 'track_name', 'playlist'])
    return df_tracks


playlist_tracks_id =get_playlist_track_ids(playlist_ids)


In [22]:
#add new columns to the df
playlist_tracks_features=pd.concat([playlist_tracks_id,pd.DataFrame(columns=['danceability', 'energy', 'key', 'loudness',  'mode', 'speechiness', 
                   'acousticness', 'instrumentalness',  'liveness', 'valence',  'tempo' ])])
#rearange columns
playlist_tracks_features = playlist_tracks_features[['track_id', 'track_name', 'artist', 'danceability', 'energy', 
                                                      'key', 'loudness',  'mode', 'speechiness', 'acousticness',
                                                      'instrumentalness',  'liveness', 'valence',  'tempo' ]]

#get audio features of a track
for i,track_id in enumerate(playlist_tracks_id['track_id']):
    track_features=spotify.get_audio_features(track_id)
    playlist_tracks_features.loc[i, ['danceability', 'energy', 'key', 'loudness',  'mode', 'speechiness', 'acousticness', 'instrumentalness',
                          'liveness', 'valence',  'tempo' ]]=[track_features['danceability'], track_features['energy'], track_features['key'],
                                                              track_features['loudness'],  track_features['mode'], track_features['speechiness'],
                                                              track_features['acousticness'], track_features['instrumentalness'], track_features['liveness'],
                                                              track_features['valence'], track_features[ 'tempo']]

In [24]:
df = pd.merge(playlist_tracks_features, playlist_tracks_id[['playlist','track_id']], on='track_id')

In [30]:
for col in df.iloc[:,3:-1]:
    #from object to float
    df[col] = df[col].astype(float)


In [31]:
df

,track_id,track_name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,playlist
0,5GXAXm5YOmYT0kL5jHvYBt,I Feel It Coming,The Weeknd,0.773,0.8190,0.0,-5.946,0.0,0.1180,0.4260,0.000000,0.0679,0.585,92.990,album_id_10s
1,6v3KW9xbzN5yKLt9YKDYA2,Señorita,Shawn Mendes,0.759,0.5480,9.0,-6.049,0.0,0.0290,0.0392,0.000000,0.0828,0.749,116.967,album_id_10s
2,3hB5DgAiMAQ4DzYbsMq1IT,Love Yourself,Justin Bieber,0.609,0.3780,4.0,-9.828,1.0,0.4380,0.8350,0.000000,0.2800,0.515,100.418,album_id_10s
3,7qiZfU4dY1lWllzX7mPBI3,Shape of You,Ed Sheeran,0.825,0.6520,1.0,-3.183,0.0,0.0802,0.5810,0.000000,0.0931,0.931,95.977,album_id_10s
4,47Slg6LuqLaX0VodpSCvPt,Just the Way You Are,Bruno Mars,0.637,0.8430,5.0,-5.413,1.0,0.0432,0.0151,0.000000,0.0876,0.434,109.012,album_id_10s
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705,6bnFjTW3EnMG4BwreDnSHK,Mr. Blue,The Fleetwoods,0.374,0.0889,4.0,-17.475,1.0,0.0356,0.9310,0.000007,0.1110,0.211,96.677,album_id_50s
706,5hznMxKe11iqwcW8BFlNWZ,"The Theme From ""A Summer Place""",Percy Faith,0.311,0.4550,0.0,-10.150,1.0,0.0311,0.2700,0.919000,0.4700,0.845,186.162,album_id_50s
707,6lYeYgSkWh6TZDQy6YZuvG,Just A Gigolo - Remastered,Louis Prima,0.525,0.4030,8.0,-11.987,1.0,0.0945,0.5970,0.000000,0.0565,0.718,125.187,album_id_50s
708,4nAJtcmiyoL0ARY5WZY9IN,Rebel Rouser,Duane Eddy,0.388,0.4340,7.0,-11.997,1.0,0.0354,0.7890,0.954000,0.7280,0.873,206.313,album_id_50s


In [32]:
df.to_csv("spotify_decades", index=False)